# I



In [2]:
import numpy as np
from sklearn.cluster import KMeans

In [3]:
# We will use assume that we have a ready implementation of KMeans algorithm for the sake of saving time

def KMeans(X, k):
    # ...
    return clusters, stds

In [4]:
class RBFNet:
    def __init__(self, X, y, tX, ty, num_of_classes, k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.k = k
        self.std_from_clusters = std_from_clusters
        
    def _eucledian(x1, x2):
        return np.linalg.norm(x1 - x2)
        
    def _rbf(self, x, c, std):
        beta = 1 / std ** 2  # it is a hyperparameter, there are other versions of it
        d = self._eucledian(x, c) ** 2
        return np.exp(-d * beta)
    
    def _one_hot(y, k):
        res = np.eye(k)[np.array(y).reshape(-1)]
        return res.reshape(list(y.shape)+[k])
        
    def fit(self):
        centroids, std_list = KMeans(self.X, self.k)

        if not self.std_from_clusters:
            dMax = np.max([self._eucledian(c1, c2) for c1 in centroids for c2 in centroids])
            std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        rbfX = np.array([
            [self.rbf(x, c, s) for (c, s) in zip(centroids, std_list)]
            for x in X
        ])

        # w = (X_T * X)^(-1) * X_T * y
        m = np.linalg.inv(np.dot(rbfX.T, rbfX))
        self.w = np.dot(np.dot(m, rbfX.T), self._one_hot(self.y, self.k))

        RBF_list_tst = np.array([
            [self._rbf(tx, c, s) for (c, s) in zip(centroids, std_list)]
            for tx in self.tX
        ])

        self.pred_ty = np.dot(RBF_list_tst, self.w)
        self.pred_ty = np.array([np.argmax(y) for y in self.pred_ty])
        return self.pred_ty
        

# II

Очевидно, дана задача - це задача на оптимізацію і перше, що мені прийшло на думку - використати генетичний алгоритм (напевне тому що я йому присвятила найбільше часу у вивченні). Втім, він тут не надто застосовний, оскільки його ціль - знайти оптимальні точки у випадку відомої функції, тут же, нам окрім даних за попередні роки нічого не відомо.

Це приводить нас до мурашиних алгоритмів. Наприклад можна скористатися алгоритмом ACO - ant colony optimisation. У основі алгоритму лежить поведінка мурашиної колонії — маркування вдалих доріг великою кількістю феромону. Робота починається з розміщення мурашок у вершинах графа (містах), потім починається рух мурашок — напрям визначається імовірнісним методом, на підставі формули:

${\displaystyle P_{i}={\frac {{l_{i}}^{q}\cdot {f_{i}}^{p}}{\sum _{k=0}^{N}{{l_{k}}^{q}\cdot {f_{k}}^{p}}}}}{\displaystyle P_{i}={\frac {{l_{i}}^{q}\cdot {f_{i}}^{p}}{\sum _{k=0}^{N}{{l_{k}}^{q}\cdot {f_{k}}^{p}}}}}$,
де:

${\displaystyle P_{i}}P_{i}$ — вірогідність переходу дорогою ${\displaystyle i}i,
{\displaystyle l_{i}}{\displaystyle l_{i}}$ — довжина ${\displaystyle i}$i-ого переходу,
${\displaystyle f_{i}}{\displaystyle f_{i}}$ — кількість феромонів на ${\displaystyle i}$i-ому переході,
${\displaystyle q}q — величина, яка визначає «жадібність» алгоритму,
{\displaystyle p}p — величина, яка визначає «стадність» алгоритму і
{\displaystyle q+p=1}{\displaystyle q+p=1}.$

Також можна звернутися до методів лінійного програмування і використати симплекс метод на даних, які у нас є.

Зрозуміло що для вищої точності результату нам необхідно більше даних.